In [ ]:
  #!/usr/bin/env python

try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
import numpy as np
from astropy.io import fits
from astropy.table import Table
import sys
sys.path.insert(0,'/home/henrique/Programas_Cosmologia/CLCosmo_Sim/cluster_abundance/')
sys.path.insert(0,'../../scripts')

import pyccl as ccl
%load_ext autoreload
%autoreload 2
import CL_COUNT_class_cluster_abundance as cl_count
import time
from nc_ccl import create_nc_obj, ccl_cosmo_set_high_prec

Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

initialize the `Cosmological model`

In [ ]:
cosmo = Nc.HICosmo.new_from_name(Nc.HICosmo, "NcHICosmoDEXcdm")
reion = Nc.HIReionCamb.new () 

prim = Nc.HIPrimPowerLaw.new () 

cosmo.add_submodel (reion)
cosmo.add_submodel (prim)

dist = Nc.Distance.new(2.0)

tf = Nc.TransferFunc.new_from_name('NcTransferFuncEH')

psml = Nc.PowspecMLTransfer.new (tf)
psml.require_kmin (1.0e-3)
psml.require_kmax (1.0e3)
psf = Ncm.PowspecFilter.new (psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0 ()

mulf = Nc.MultiplicityFuncTinker.new ()
mulf.set_mdef (Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta (200.0)
#mulf.set_sim (Nc.MultiplicityFuncBocquetSim.DM)

hmf = Nc.HaloMassFunction.new (dist, psf, mulf)
hmf.set_area_sd(5264.05)

#Creating the halo bias 
hbias_Tinker = Nc.HaloBiasTinker.new(hmf)

# Define proxy modelling
Use a mass proxy, define the probability for observing a proxy given a mass and redhsift

$$
P(\log\lambda|M,z) = N(\mu(M,z), \sigma^2(M,z))
$$
the mean is
$$
    \mu(M,z) = \mu_0 + a_\mu^M\log_{10}\frac{M}{M_0} + a_\mu^z\log_{10}\frac{1+z}{1+z_0} 
$$
variance is
$$
\sigma(M,z) = \sigma_0 + a_\sigma^M\log_{10}\frac{M}{M_0} + a_\sigma ^z\log_{10}\frac{1+z}{1+z_0} 
$$

In [ ]:
cluster_z = Nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (0,2))
cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassAscaso{'M0':<%20.15e>,'z0':<%20.15e>,'lnRichness-min':<%20.15e>, 'lnRichness-max':<%20.15e>}" % (3e14/(0.71),0.6, 0, 1/np.log(10) * np.log10(10e2)))
cluster_m.param_set_by_name('mup0', 3.19)
cluster_m.param_set_by_name('mup1', 2/np.log(10))
cluster_m.param_set_by_name('mup2', -0.7/np.log(10))
cluster_m.param_set_by_name('sigmap0', 0.33)
cluster_m.param_set_by_name('sigmap1', -0.08/np.log(10))
cluster_m.param_set_by_name('sigmap2', 0/np.log(10))

Create the `Cluster Abundance Data`

In [ ]:
ca = Nc.ClusterAbundance.new(hmf,hbias_Tinker)
mset = Ncm.MSet.new_array([cosmo,cluster_m,cluster_z])

#Generates cluster count catalog data


ncdata = Nc.DataClusterNCount.new(ca,'NcClusterRedshiftNodist','NcClusterMassAscaso')
rng = Ncm.RNG.pool_get("example_ca_sampling");
ncdata.init_from_sampling(mset,5264.05*(np.pi/180)**2,rng)
ncdata.catalog_save("ca_mean_bias_ascaso.fits", True)

ncdata_fits = fits.open('ca_mean_bias_ascaso.fits')
#ncdata_fits.info()

ncdata_data =  ncdata_fits[1].data

ncdata_Table = Table(ncdata_data)
ncdata_Table.columns
ncdata_Table.sort('LNM_OBS')
mset.pretty_log()


# Un-binned approach

$$
b_i =\frac{\Omega_s}{x_i}\int_{\log_{10}M_{min}}^{\log_{10}M_{max}}\frac{d^2V(z_i)}{dz d\Omega}\frac{dn(M,z_i)}{d\log_{10}M}P(\lambda_i|M,z_i)b(M,z_i) d\log_{10}M
$$

In [ ]:
display(ncdata_Table)
ca.prepare(cosmo,cluster_z,cluster_m)


print(5264.05*(np.pi/180)**2*ca.mean_bias(cosmo, cluster_z, cluster_m)/ca.n(cosmo,cluster_z,cluster_m))
dnbias = []
dv = []

for cluster in ncdata_Table:
    dnbias.append(5264.05*(np.pi/180)**2*ca.intp_d2n_bias(cosmo, cluster_z, cluster_m, [cluster[1]], None, [cluster[0]], None) / ca.lnM_p_d2n(cosmo, cluster_z, cluster_m, [cluster[1]],None, cluster[0]))

dnbias


In [ ]:
plt.figure(figsize = (7,5))
plt.scatter(ncdata_Table['Z_OBS'], dnbias ,s = 7, c = ncdata_Table['LNM_OBS'], cmap = 'viridis')
plt.colorbar()
plt.xlabel('redshift', fontsize = 20)
plt.ylabel('$b_i$', fontsize = 20)
#plt.yscale('log')
plt.grid(True, which="both")
plt.show()


plt.figure(figsize = (7,5))
plt.scatter(ncdata_Table['Z_OBS'], dnbias ,s = 7, c = ncdata_Table['LNM_TRUE'], cmap = 'viridis')
plt.colorbar()
plt.xlabel('redshift', fontsize = 20)
plt.ylabel('$b_i$', fontsize = 20)
#plt.yscale('log')
plt.grid(True, which="both")
plt.show()

# Binned approach

Compute the integral

$$
b_{\alpha\beta}^{\rm predicted} = \frac{\Omega_s}{N_{\alpha\beta}^{\rm predicted}}\int_{z_\alpha}^{z_{\alpha + 1}} dz\int_{\lambda_{\beta}}^{\lambda_{\beta + 1}}d\lambda\int_{\log_{10}M_{min}}^{\log_{10}M_{max}}\frac{d^2V(z)}{dz d\Omega}\frac{dn(M,z)}{d\log_{10}M}b(M,z)P(\lambda|M,z)d\log_{10}M
$$

In [ ]:
#Bins definition
z_edges = np.linspace(0.25, 2, 5)
proxy_edges = np.linspace(0, 2, 4)

In [ ]:
#Using Numcosmo

t0 = time.time()
ca.prepare(cosmo,cluster_z,cluster_m)
N_proxy_numcosmo = []


for rich_l, rich_u in zip(proxy_edges, proxy_edges[1:]):
    N_zbin = []
    for zl, zu in zip(z_edges,z_edges[1:]):
        N_zbin.append(5264.05*(np.pi/180)**2*ca.intp_bin_d2n_bias(cosmo,cluster_z,cluster_m, [rich_l], [rich_u],None ,[zl],[zu], None)/ca.intp_bin_d2n(cosmo, cluster_z, cluster_m, [rich_l], [rich_u], None, [zl], [zu], None))
    
    N_proxy_numcosmo.append(N_zbin)

t1 = time.time()
print('time (seconds) = ' + str(t1-t0))


In [ ]:
plt.figure(figsize = (9,10))
plt.yscale('log')
z_ = [np.mean([z_edges[i],z_edges[i+1]]) for i in np.arange(len(z_edges)-1)]
proxy = [np.mean([proxy_edges[i],proxy_edges[i+1]]) for i in np.arange(len(proxy_edges)-1)]


plt.tick_params(axis='both', which="both", labelsize= 15)
for f in range(len(proxy_edges)-1):
    if f == 0:
        label = []
    y_simps = (N_proxy_numcosmo[:][f])
    plt.scatter(z_, y_simps, label = str(f+1)+' richness bin')
plt.grid(True, which="both" )
plt.ylabel(r'$b_{\alpha \beta}$', fontsize = 20)
#plt.ylim(1,2)
plt.xlabel(r'$redshift$', fontsize = 20)
plt.legend(frameon = False, fontsize = 20)